In [133]:
# # -*- coding: utf-8 -*-
# from ckiptagger import data_utils # first use ckip, need to download data file
# data_utils.download_data_gdown("./")
from ckiptagger import WS, POS, NER, construct_dictionary

ws = WS("./ckip_data") # word segment
# pos = POS("./ckip_data") 
# ner = NER("./ckip_data") # entity recognition


import pandas as pd
import numpy as np
import time
import math
import re
import demoji
# demoji.download_codes()
from selenium.webdriver import ActionChains # activate esc key
from selenium.webdriver.common.keys import Keys # activate esc key

#====== import jieba dict =====
import jieba
jieba.set_dictionary('dataset/dic_trad/dict.txt.big')
jieba.load_userdict('dataset/dic_trad/self_define_dict.txt')


#====== selenium cathc NoSuchElementException ======
from selenium.common.exceptions import NoSuchElementException   
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

#====== evaluation metrics ======
from sklearn.metrics import mean_squared_error #均方誤差
from sklearn.metrics import mean_absolute_error as mae #平方絕對誤差
from sklearn.metrics import r2_score


Building prefix dict from /home/alvin/essay/dataset/dic_trad/dict.txt.big ...
Loading model from cache /tmp/jieba.ucbd952ca8a83da26cb61ea55c46a2e3b.cache
Loading model cost 0.745 seconds.
Prefix dict has been built successfully.


In [2]:
#====== get antusd senti dict ======
colNames = ['Word','Score','Pos','Neu','Neg','Non','Not']
antusd = pd.read_csv('dataset/ANTUSD_traditional/opinion_word.csv', names = colNames) 
word_senti_score = dict(zip(antusd['Word'],antusd['Score'])) # find score via word, 27221 words

# print(antusd)
# print(word_senti_score)

In [83]:
#====== emoji_sentiment_dict ======
emoji_sentiment = pd.read_csv('dataset/dic_trad/Emoji_Sentiment.csv') # read Emoji_Sentiment.csv
emoji_sentiment.drop(columns=['Unnamed: 0'], inplace = True)
emoji_sentiment_dict = dict(zip(emoji_sentiment['Emoji'],emoji_sentiment['Sentiment score'])) # key,value

In [4]:
def get_selenium_param(filename):
    paramList = []
    with open("dataset/ig_article/"+filename, "r") as f:
        for line in f:
            single_store_param = line.split(',')
            single_store_param[2] = single_store_param[2].replace('\n','')
            paramList.append(single_store_param)

    return paramList




In [86]:
#====== extend senti. dict ======
def self_extend_senti_dict(word_senti_score):
    with open("dataset/dic_trad/extend_senti_score_dict.txt", "r") as f:
        for line in f:
            senti_word = line.split()[0]
            senti_score = float(line.split()[1])
            word_senti_score[senti_word] = senti_score


#====== extend senti. dict ======
def self_extend_senti_dict_hotpot(word_senti_score):
    with open("dataset/dic_trad/extend_senti_score_dict_hotpot.txt", "r") as f:
        for line in f:
            senti_word = line.split()[0]
            senti_score = float(line.split()[1])
            word_senti_score[senti_word] = senti_score    
        
#====== add emoji senti score ======
def add_emoji_senti_score(word_senti_score,emoji_sentiment_dict):
    word_senti_score.update(emoji_sentiment_dict)
#     return word_senti_score





word_senti_score['good'] = 0.0550611
word_senti_score['棒'] = 0.1
word_senti_score['中規中矩'] = -0.05
word_senti_score['感覺'] = 0.02
word_senti_score['覺得'] = 0.02
word_senti_score['第一'] = 0.2
word_senti_score['虛'] = -0.2
word_senti_score['爛'] = -0.2
word_senti_score['差'] = -0.2
word_senti_score['好差'] = -0.2
word_senti_score['久'] = -0.1
word_senti_score['太久'] = -0.4
word_senti_score['厲害'] = 0.2
word_senti_score['熱死'] = -0.2
word_senti_score['排隊'] = -0.1
word_senti_score['不高'] = -0.3
word_senti_score['鹹'] = -0.1
word_senti_score['好鹹'] = -0.2
word_senti_score['太鹹'] = -0.3
word_senti_score['沒'] = -0.1
word_senti_score['普通'] = -0.02
word_senti_score['不'] = -0.05
self_extend_senti_dict(word_senti_score)
self_extend_senti_dict_hotpot(word_senti_score)
add_emoji_senti_score(word_senti_score, emoji_sentiment_dict)

In [88]:
emoji_sentiment_dict

{'😂': 0.221,
 '❤': 0.746,
 '♥': 0.657,
 '😍': 0.6779999999999999,
 '😭': -0.09300000000000001,
 '😘': 0.701,
 '😊': 0.644,
 '👌': 0.563,
 '💕': 0.632,
 '👏': 0.52,
 '😁': 0.449,
 '☺': 0.657,
 '♡': 0.669,
 '👍': 0.521,
 '😩': -0.368,
 '🙏': 0.41700000000000004,
 '✌': 0.46299999999999997,
 '😏': 0.332,
 '😉': 0.46299999999999997,
 '🙌': 0.5589999999999999,
 '🙈': 0.43200000000000005,
 '💪': 0.555,
 '😄': 0.42100000000000004,
 '😒': -0.374,
 '💃': 0.7340000000000001,
 '💖': 0.7120000000000001,
 '😃': 0.557,
 '😔': -0.146,
 '😱': 0.19,
 '🎉': 0.738,
 '😜': 0.455,
 '☯': 0.001,
 '🌸': 0.65,
 '💜': 0.654,
 '💙': 0.73,
 '✨': 0.35100000000000003,
 '😳': 0.018000000000000002,
 '💗': 0.657,
 '★': 0.28300000000000003,
 '█': -0.032,
 '☀': 0.465,
 '😡': -0.17300000000000001,
 '😎': 0.491,
 '😢': 0.006999999999999999,
 '💋': 0.691,
 '😋': 0.631,
 '🙊': 0.45899999999999996,
 '😴': -0.08,
 '🎶': 0.537,
 '💞': 0.7390000000000001,
 '😌': 0.48200000000000004,
 '🔥': 0.139,
 '💯': 0.12,
 '🔫': -0.19399999999999998,
 '💛': 0.7090000000000001,
 '💁': 0

In [84]:
print(len(word_senti_score),len(emoji_sentiment_dict))

27367 751


In [ ]:
#====== test ckip word segment ======

# text = '傅達仁今將執行安樂死，卻突然爆出自己20年前遭緯來體育台封殺，他不懂自己哪裡得罪到電視台。'
# text = '"鬼金棒，我就跟！就這樣，我又跟我的前衣食父母吃飯！連吃兩天拉麵🍜一個胖的節奏#鬼金棒拉麵#辣正常麻增量#結果真的麻到我頭皮發麻 我到底要不要改機票呢😏"'
# text = '期盼那麼久才吃到，果然不負期望，無論是湯頭還是肉片都是最佳狀態，太享受了'
text = '好久不見！，體力一年不如一年，#20191101，服務態度超優的，海鮮😋,海底撈圈字遊戲送一盤肉,👍👍👍撈撈撈，💋💋💋,排隊排好久 超值得, 我要再來刷二訪 😋， 以愛吃重口味的我來說 麵條的硬度很讚 乾, 想吃很膩害的溏心蛋 這家沾麵真的很優,湯頭也很濃 我不愛沾麵 難波萬台灣美食成為我的愛店之一👍🏻好多人，不知道要排多久,麻的味道超級棒,推推推,湯頭還是很棒，豆芽菜的鑊氣沒了很難過，大概一陣子不會回訪了,good excelent,熱死第一名,可惜還是沒把湯喝完,應該很普通,不是我的最愛。'


ws_results = ws([text])
# pos_results = pos(ws_results)
# ner_results = ner(ws_results, pos_results)

print(ws_results)
# print(pos_results)
# for name in ner_results[0]:# print entity
#     print(name)

In [82]:
list(zip(emoji_sentiment['Emoji'],emoji_sentiment['Sentiment score']))

[('😂', 0.221),
 ('❤', 0.746),
 ('♥', 0.657),
 ('😍', 0.6779999999999999),
 ('😭', -0.09300000000000001),
 ('😘', 0.701),
 ('😊', 0.644),
 ('👌', 0.563),
 ('💕', 0.632),
 ('👏', 0.52),
 ('😁', 0.449),
 ('☺', 0.657),
 ('♡', 0.669),
 ('👍', 0.521),
 ('😩', -0.368),
 ('🙏', 0.41700000000000004),
 ('✌', 0.46299999999999997),
 ('😏', 0.332),
 ('😉', 0.46299999999999997),
 ('🙌', 0.5589999999999999),
 ('🙈', 0.43200000000000005),
 ('💪', 0.555),
 ('😄', 0.42100000000000004),
 ('😒', -0.374),
 ('💃', 0.7340000000000001),
 ('💖', 0.7120000000000001),
 ('😃', 0.557),
 ('😔', -0.146),
 ('😱', 0.19),
 ('🎉', 0.738),
 ('😜', 0.455),
 ('☯', 0.001),
 ('🌸', 0.65),
 ('💜', 0.654),
 ('💙', 0.73),
 ('✨', 0.35100000000000003),
 ('😳', 0.018000000000000002),
 ('💗', 0.657),
 ('★', 0.28300000000000003),
 ('█', -0.032),
 ('☀', 0.465),
 ('😡', -0.17300000000000001),
 ('😎', 0.491),
 ('😢', 0.006999999999999999),
 ('💋', 0.691),
 ('😋', 0.631),
 ('🙊', 0.45899999999999996),
 ('😴', -0.08),
 ('🎶', 0.537),
 ('💞', 0.7390000000000001),
 ('😌', 0.4820

In [6]:
# word_pos = dict(zip(ws_results[0],pos_results[0])) # combine word and pos tag

In [7]:
#====== data preprocess ======
def data_preprocess(articles): 
    # only get string comment & preprocess fans, likes
    all_comments, assigned_index, fans_count, likes  = [],[],[],[]  # assigned_index: denote the original dataframe index of comments
    for i in range(len(articles['text'])):     # remove '\n'
        if type(articles['text'][i]) == str:   # remove non-string
            if articles['text'][i].find('\n'): # if contain '\n', replace with ','
                all_comments.append(articles['text'][i].replace('\n','，'))
            else:
                all_comments.append(articles['text'][i])
            assigned_index.append(i)
            
#             # get assigned fans_count
#             if type(articles['fans_count'][i]) == np.float64 and math.isnan(articles['fans_count'][i]):
#                 fans_count.append(0)
#             elif type(articles['fans_count'][i]) == np.float64 and np.isnan(articles['fans_count'][i]):
#                 fans_count.append(0)
#             elif str(articles['fans_count'][i]).find('萬') != -1:
#                 wan_index = articles['fans_count'][i].find('萬')
#                 transform_count = (float(articles['fans_count'][i][:wan_index])) * 10000
#                 fans_count.append(transform_count)
#             else:
#                 fans_count.append(int(articles['fans_count'][i]))
            fans_count = []


            # get assigned likes
            temp_like = articles['likes'][i]
            if type(temp_like) == np.float64 or type(temp_like) == np.int64:
                likes.append(temp_like)
            elif type(temp_like) == str:
                if temp_like.isdigit():
                    likes.append(float(temp_like))
                else:
                    likes.append(1)
            else:
                likes.append(temp_like)
            

    
    return all_comments, assigned_index, fans_count, likes

In [8]:
#====== cal senti score by jieba ======
jieba.load_userdict('dataset/dic_trad/self_define_dict.txt') # load self-define dict

def get_senti_score(text):
    '''calculate one sentence's score'''
    ws_results = [jieba.lcut(text)] # word segment
    # get sentiword in dict
    total_score = 0
    for word in ws_results[0]:
        if word in word_senti_score:
            score = word_senti_score[word]
            total_score += score 
            print(word,score) # print each senti word's senti score
    
    
    print('sent_score:',total_score)
    if total_score >= 1:
        return 1
    elif total_score <= -1:
        return -1
    else:
        return total_score

In [ ]:
#====== cal senti score by ckip ======
# def get_senti_score(text):
#     ws_results = ws([text]) # word segment
#     total_score = 0
#     for word in ws_results[0]:
#         if word in word_senti_score:
#             score = word_senti_score[word]
#             total_score += score 
#             print(word,score) # print each senti word's senti score
    
    
#     print('sent_score:',total_score)
#     if total_score >= 1:
#         return 1
#     elif total_score <= -1:
#         return -1
#     else:
#         return total_score

In [9]:
#====== get each comment's score ======
def get_each_comment_score(all_comments):
    all_comments_scores = []
    for single_comment in all_comments:
        print('\ncomment:',single_comment)
        all_comments_scores.append(get_senti_score(single_comment))
        
    return all_comments_scores

In [10]:
#====== cal store score ======      # not yet consider no comment post's effect (maybe effective) 
def get_store_score(weight, each_comment_score, assigned_index):
    '''store score = comment score * likes(weight) /all likes'''
    weighted_score = 0
    total_weight = 0
    for i in range(len(each_comment_score)):
        if not np.isnan(weight[i]):
            weighted_score += (weight[i] * each_comment_score[i])
            total_weight += weight[i] 
            
    store_score = weighted_score / total_weight

    return store_score


In [11]:
def output_csv(filename, data_list, write_method):

    import csv

    # 開啟輸出的 CSV 檔案
    with open(filename, write_method, newline='') as csvfile:
        writer  = csv.writer(csvfile)   # 建立 CSV 檔寫入器
        for row in data_list: # format: two dimension
            writer.writerow(row)
            

In [ ]:
#====== remove foreign language post ======
for i,content in zip(articles[articles['text'].notnull()]['text'].index,
                     articles[articles['text'].notnull()]['text']):
    if not re.search(r'[\u4e00-\u9fff]+', content) and not demoji.findall(content): # check contain chinese
        print(i,content)
        # drop assigned row
        articles.drop(i, inplace=True) 
        
# reset dataframe index
articles.reset_index(drop=True) 

In [15]:
text = '這是一場五感都被放大的饗宴。﻿，﻿，有硬度及Q勁的麵體，🐠很喜歡，搭配辣少麻少的濃厚湯頭，非常過癮！鬼金棒的辣和麻是一流的，入口先感受到辣，隨之而來的麻會讓你感受到舌尖的震顫，建議第一次吃、不是很敢吃麻辣的朋友跟🐠一樣選擇辣少麻少。﻿，﻿，鬼金棒的拉麵，就是這樣強勁有力。﻿，﻿，叉燒肉也和大多數的拉麵店不同，選用大塊的五花肉，乍看之下有點像控肉（但絕對不是），肥瘦相間、軟嫩好入口，有別於一般薄片叉燒，鬼金棒就是這樣的霸氣！﻿，﻿，最後額外提一下，店裡的冰開水不知道為什麼，特別甘甜好喝啊🤣﻿，﻿，﻿，-·-·-·-·-·-﻿，🔺用餐請遵守店內顧客須知，大家一起當優良好客人！﻿，🔺吃完麵再加白飯100分！但麵的份量本身就蠻大了，要自己斟酌哦！﻿，——————————，﻿，🌟如果喜歡濃厚拉麵，或者吃膩了一般口味的拉麵，來鬼金棒走一遭吧☺️﻿，﻿，——————————，🏠辣麻味噌拉麵 鬼金棒﻿，📍104台北市中山區長安西路19巷2弄35號（近捷運中山站）﻿，🕐平日11:30-14:00/17:00-20:30，假日11:30-20:30﻿，，#辣麻味噌拉麵 #鬼金棒 #鬼金棒カラシビ味噌ラーメン #拉麵 #叉燒肉 #捷運美食 #捷運中山站 #ramen #ramennoodles 當然是想到鬼金棒💕，麻的味道超級棒，讓你一口接一口🤤，肉的份量也很足👍，湯頭非常濃郁‼️，吃#nightmarket #taiwan #taipei #noodles #stinkytofu 🤤#squid #seafood #dumplings #ningxia #nightlife,豪豪ㄘ 嘴唇超麻～～～,敲好ㄘ🌶🌶🌶🌶🍜'

# print(ws([text]))
demoji.findall(text)
for emoji in demoji.findall(text):
    if emoji_sentiment_dict.get(emoji):
        print(emoji,emoji_sentiment_dict[emoji])

🌟 0.327
🐠 0.414
🏠 0.5
👍 0.521
💕 0.632
🍜 0.395
📍 0.111


In [13]:
jieba.lcut('喝起來更香濃愛吃重口味的我來說嗚嗚嗚嗚可惜吃到後面有點膩今天沾麵日！白湯&海湯沾麵🍜')

['喝',
 '起來',
 '更',
 '香濃',
 '愛吃',
 '重',
 '口味',
 '的',
 '我',
 '來說',
 '嗚嗚',
 '嗚嗚',
 '可惜',
 '吃',
 '到',
 '後面',
 '有點',
 '膩',
 '今天',
 '沾麵',
 '日',
 '！',
 '白湯',
 '&',
 '海湯',
 '沾麵',
 '🍜']

In [844]:
# len(all_comments_scores)

50

In [771]:
# articles[(articles['text']).isnull()]['text'] # get NaN text in article

3     NaN
8     NaN
15    NaN
19    NaN
26    NaN
27    NaN
37    NaN
38    NaN
Name: text, dtype: object

In [16]:
# all_comments_scores

In [100]:
#====== cal one store rating ======

#====== update senti_dict & jieba_dict ======
self_extend_senti_dict(word_senti_score)
jieba.load_userdict('dataset/dic_trad/self_define_dict.txt') # load self-define dict


# get articles, followers
# fileName = 'hotpot/article_問鼎麻辣鍋忠孝店' # 147
fileName = 'hotpot/article_滿堂紅bellavita店_松江店' # 101



# fileName = 'article_山嵐拉麵_公館店_台灣總店' # 85 post
# fileName = 'article_山嵐拉麵' # 33 post (remove)
# fileName = 'article_你回來啦 Okaeri 拉麵'# 96
# fileName = 'article_鬼金棒_中山北路' # 126
# fileName = 'article_麵屋壹の穴 ICHI' # 50 (remove)
# fileName = 'article_麵屋壹の穴 ICHI_麵屋壹の穴ichi-沾麵專門店' # 106
# fileName = 'article_柑橘 Shinn' # 69


articles = pd.read_csv(open('dataset/ig_article/store_article/'+fileName+'.csv')) 

# remove '\n', delete non-string
all_comments, assigned_index, fans_count, likes = data_preprocess(articles)  

# get each comment's score
all_comments_scores = get_each_comment_score(all_comments)

# get store rating
# print('\nstore final score:',get_store_score(fans_count, all_comments_scores, assigned_index)) # weight: fans
store_rating = get_store_score(likes, all_comments_scores, assigned_index)
print('\nstore final score:',store_rating) # weight: likes
print('post count:',len(all_comments_scores))

# output ramen rating 
output_csv('ramen_rating.csv', [[fileName, store_rating]], 'a+')




comment: 老規矩，年紀最大的付錢... 可以跟我講英文啊，我真的聽得懂啦🥺，，吃好吃飽吃滿🇰🇷🇹🇼 #謝謝韓國燒酒 ，#chiuaufearless #family #hotpot #korea #taiwan #燒酒 #dinner #20191031
最大 0.0
可以 0.1551797
好吃 0.37684395
🇷 0.667
🇹 0.579
🇼 0.33299999999999996
謝謝 0.2974824
sent_score: 2.4085060499999997

comment: 正中午被拉去吃麻辣，#5人吃到飽，#好會點啊，#超飽晚餐免了🤪🤪，#滿堂紅
sent_score: 0

comment: 一年一聚，甚是想念！
sent_score: 0

comment: 📍台北信義區｜BELLAVITA寶麗廣塲﻿，﻿，🍲滿堂紅頂級麻辣鴛鴦鍋﻿，單人鴛鴦鍋吃到飽 🅝🅣$798﻿，﻿，湯底7選2﻿，✨招牌麻辣鍋（小辣）﻿，泰式酸辣鍋﻿，日式昆布鍋﻿，✨牛奶味噌鍋﻿，大漠孜然鍋﻿，蘆薈養生鍋﻿，特色酸白菜鍋﻿，﻿，🔸附吃到飽菜單於圖末﻿，🔸依照食材分為：$798/$888/$1088吃到飽﻿，🔸金額皆已包含10%服務費﻿，🔸Häagen-Dazs、Cold Stone、Glacio冰淇淋吃到飽﻿，🔸日本Sapporo Premium生啤酒無限暢飲﻿，﻿，-----------------------------------------------﻿，﻿，這天來到位於貴婦百貨B2的滿堂紅鴛鴦鍋吃到飽﻿，恰巧還碰到期間限定的Bear Kingdom真的超療癒﻿，到1/6都還看得到哦！幸好還來得及生出這篇文☺️﻿，﻿，滿堂紅每間分店提供的肉品似乎都略有不同﻿，這天造訪的BELLAVITA店，11/1才剛推出全新菜單﻿，依照食材分為三種價位：$798、$888、$1088吃到飽﻿，金額都已經包含服務費！會有種比較便宜的錯覺﻿，友人覺得$798就很夠吃了，我們就沒有直上最高價﻿，湯底的部份選擇招牌麻辣鍋和牛奶味噌鍋﻿，畫完第一輪等上菜的同時，出發巡視自助吧區👣﻿，﻿，我們剛好被安排在最靠近生啤酒機的座位﻿，無奈晚上還要上班，只能忍痛和SAPPORO說88﻿，飲料的選擇非常多，有可樂機、無糖綠茶、冰沙﻿，各式沖泡茶包、雀巢咖啡機，甚至還有現磨咖啡機﻿，

In [129]:
#====== MSE ======
# input store name: [a,b,c]
# output: [score.....]
def cal_all_store_rating(store_names):
    all_store_rating = []
    all_store_post_count = [] # store info dict.
    for i,name in enumerate(store_names[1:]):
        fileName = '/article_' + name
        articles = pd.read_csv(open('dataset/ig_article/store_article/'+store_names[0]+fileName+'.csv')) 

        # remove '\n', delete non-string
        all_comments, assigned_index, fans_count, likes = data_preprocess(articles)  

        # get each comment's score
        all_comments_scores = get_each_comment_score(all_comments)

        # get store rating
        # print('\nstore final score:',get_store_score(fans_count, all_comments_scores, assigned_index)) # weight: fans
        store_rating = get_store_score(likes, all_comments_scores, assigned_index)
        print('\nstore final score:',store_rating) # weight: likes
        all_store_rating.append(store_rating)
#         all_store_post_count.append(len())
        
    return all_store_rating



#===================================== main function =======================================
# ramen
store_names = ['ramen','鬼金棒_中山北路','雞吉君拉麵','山嵐拉麵_公館店_台灣總店','你回來啦 Okaeri 拉麵',
               '麵屋壹慶','勝王','五之神製作所','柑橘 Shinn','麵屋壹の穴 ICHI_麵屋壹の穴ichi-沾麵專門店']

# hotpot
# store_names = ['hotpot',
#                '辛殿麻辣鍋',
#                '海底撈火鍋',
#                '老四川巴蜀麻辣燙-南京店',
#                '這一鍋皇室秘藏鍋物',
#                '詹記麻辣火鍋 敦南店',
#                '新馬辣武昌店_新馬辣經典馬辣鍋',
#                '青花驕 麻辣鍋',
#                '馬辣頂級麻辣鴛鴦火鍋 漢口店',
#                '問鼎麻辣鍋忠孝店',
#                '滿堂紅bellavita店_松江店']




#====== update senti_dict & jieba_dict ======
self_extend_senti_dict(word_senti_score) 
jieba.load_userdict('dataset/dic_trad/self_define_dict.txt') # load self-define dict


# cal_all_store_rating
all_store_rating = cal_all_store_rating(store_names)


print(all_store_rating)



comment: 敲好ㄘ🌶🌶🌶🌶🍜
敲好 0.4
ㄘ 0.15
🌶 0.1
🌶 0.1
🌶 0.1
🌶 0.1
🍜 0.15
sent_score: 1.0999999999999999

comment: #🍜#ラーメン#ラーメン大好き#食べ物#カラシビ，如今本寶寶也是嚐過黑碗的人了！，（有玉米筍增量）
🍜 0.15
大好 0.43823609999999996
增量 0.2
sent_score: 0.7882361

comment: 豪豪ㄘ 嘴唇超麻～～～
豪豪 0.25
ㄘ 0.15
sent_score: 0.4

comment: 2019.12.6，第二桌！，起步～走！，，#302r4b1c ，#TY ，#秉鈞的築夢人生
起步 0.0886419
築夢 0.3
sent_score: 0.3886419

comment: #nightmarket #taiwan #taipei #noodles #stinkytofu 🤤#squid #seafood #dumplings #ningxia #nightlife
🤤 0.15
sent_score: 0.15

comment: 這次選擇辣小量、麻普通，下次可以吃辣普通、麻普通～，，#台北美食 #排隊美食 #拉麵 #鬼金棒#麻辣#中山站美食
選擇 0.0210186
普通 -0.08
下次 0.1
可以 0.1551797
普通 -0.08
普通 -0.08
美食 0.16
排隊 -0.1
美食 0.16
拉麵 0.15
鬼金棒 0.25
美食 0.16
sent_score: 0.8161983

comment: 吃那個肥肉好爽
肥肉 -0.2
好爽 0.5
sent_score: 0.3

comment: Kikanbo 鬼金棒(きかんぼう)の味噌ラーメンが激辛でうまい！鬼金棒辣麻味增拉麵，麵條Q湯頭棒，直接挑戰辣麻增量過關，好吃過癮！下回可以再來挑戰辣麻鬼增量，讚！^^
鬼金棒 0.25
鬼金棒 0.25
拉麵 0.15
棒 0.13
直接 0.326906
挑戰 0.0
增量 0.2
過關 0.0051143000000000004
好吃 0.37684395
過癮 0.35
可以 0.1551797
挑戰 0.0
增量 0.2
讚 0.25
sent_score: 2.644043

👉 0.39
濃厚 0.3
👉 0.39
自助 0.4579737
沒有 -0.0921865
可以 0.1551797
服務 0.1
服務 0.1
即可 0.2236185
可以 0.1551797
自取 0.0
等待 0.2913396
到來 0.0445837
即可 0.2236185
🍴 0.537
小時 0.0
左右 0.0
沒 -0.1
排隊 -0.1
人潮 0.0
😳 0.018000000000000002
第一次 0.036000533
💕 0.1
清澈 0.4281769
濃厚 0.3
濃 0.1
味道 0.0
偏重 0.03821835
喜歡 0.875
🐷 0.368
叉燒 0.0
驚為 0.0
好吃 0.37684395
💯 0.12
鹹 -0.1
香氣 0.2
👍 0.25
吃不下 -0.514206433
XD 0.1
建議 0.0
早點 0.0
排隊 -0.1
以免 -0.07767475
😂 0.221
不論 -0.5304674
拿取 0.0
服務 0.1
收取 0.045369
排隊 -0.1
趕快 0.21264125
享用 0.0
💕 0.1
追蹤 0.0
👉 0.39
❤ 0.746
☝ 0.309
歡迎 0.475
加入 0.0625365
追蹤 0.0
🐷 0.368
發洩 0.0
拉麵 0.15
拉麵 0.15
拉麵控 0.1
🍜 0.15
美食 0.16
美食 0.16
美食 0.16
🇹 0.579
🇼 0.33299999999999996
🇹 0.579
🇼 0.33299999999999996
美食 0.16
sent_score: 11.297744800000002

comment: 拉麵！，，#雞吉君拉麵
拉麵 0.15
拉麵 0.15
sent_score: 0.3

comment: 2019-12-08，#雞吉君拉麵，#吃拉麵從板橋到內湖有點遠，#美麗華摩天輪，#冰火菠蘿油，#最近很紅的板橋菠蘿包蛋塔，#休假就是吃吃吃
拉麵 0.15
拉麵 0.15
美麗 0.6160439
sent_score: 0.9160439

comment: #內湖美食 #雞吉君拉麵，位在捷運西湖站1號出口左轉，內湖路一段285巷直直走到環山路就到了，從2017年開業至今 #還在排隊，所以也去排一下（這是什麼概

🌑 0.45799999999999996
🌕 0.556
🌕 0.556
🌕 0.556
🌕 0.556
🌑 0.45799999999999996
值得 0.23
🌕 0.556
🌕 0.556
🌕 0.556
🌗 0.611
🌑 0.45799999999999996
意願 0.35227129999999995
🌕 0.556
🌕 0.556
🌕 0.556
🌘 0.579
🌑 0.45799999999999996
拉麵 0.15
山嵐拉麵 0.25
山嵐 0.25
拉麵 0.15
美食 0.16
美食 0.16
拉麵 0.15
拉麵 0.15
拉麵 0.15
叉燒 0.0
ramen 0.15
sent_score: 16.229251116000007

comment: 第一次吃到叉燒肉這麼大塊的拉麵🍜姊嚇到了，湯頭好麵Q、叉燒Q彈厚實，服務又好親切，整個值得再來的拉麵🍜，#raman #山嵐 #山嵐拉麵#taipeifood
第一次 0.036000533
叉燒 0.0
拉麵 0.15
🍜 0.15
叉燒 0.0
厚實 0.41266535
服務 0.1
親切 0.5618249
值得 0.23
拉麵 0.15
🍜 0.15
山嵐 0.25
山嵐拉麵 0.25
sent_score: 2.4404907829999996

comment: We are eating good in Taipei. #taipei #sushi #taiwan #food #expat #travel #foodie
good 0.0550611
sent_score: 0.0550611

comment: 吃= = 山嵐拉麵，是還可以，但感覺有點過譽，#豐盛白湯拉麵
山嵐拉麵 0.25
可以 0.1551797
感覺 0.02
豐盛 0.2
白湯 0.15
拉麵 0.15
sent_score: 0.9251797000000002

comment: No.............it’s Monday again...😫，#taiwan #taiwanfood #taipei #taipeifood #taipeifoodies #台北美食
😫 -0.145
美食 0.16
sent_score: 0.015000000000000013

comment

拉麵 0.15
總是 0.05
討厭 -0.26391
長大 0.0
知道 0.529169
美妙 0.725
一直 0.2729052
想 0.005
經過 0.1523594
圓夢 0.0474995
喜歡 0.875
拉麵 0.15
看到 0.0445837
介紹 0.05
知道 0.529169
可能 0.0
不會 -0.27884699999999996
叉燒 0.0
不會 -0.27884699999999996
肥肉 -0.2
不會 -0.27884699999999996
膩 -0.2
蔬菜 0.0
沒有 -0.0921865
不過 0.0
已經 0.029755599999999997
不是 -0.7609348000000001
重點 0.1391674
謝謝 0.2974824
可以 0.1551797
拉麵 0.15
鹹 -0.1
🍜 0.15
景點 0.0
美食 0.16
美食 0.16
推薦 0.019754
景點 0.0
美食 0.16
拉麵 0.15
拉麵 0.15
拉麵控 0.1
拉麵 0.15
🍜 0.15
拉麵 0.15
ramen 0.15
sent_score: 3.8484525999999994

comment: 比預期的晚下班 還以為吃不到了，有嚼勁的粗麵加上厚片叉燒整個很對味，蒜味豚骨真的超開胃，雖然吃完嘴巴超臭😅，#濃蒜豚骨拉麵
預期 0.1487412
不到 -0.4717674
加上 0.16797105
叉燒 0.0
對味 0.0
😅 0.17800000000000002
拉麵 0.15
sent_score: 0.17294484999999998

comment: #，國父紀念館每次都排很多人的#你回來啦，某天平日晚上去一探究竟 幸運只等了十分鐘，#濃厚豚骨拉麵 還真的沒在客氣的濃厚，這裡的口味真的是很日式的那種拉麵 濃到湯頭都略微稠稠的那種 口味不錯 份量很大 叉燒也好吃，但湯頭對我來說真的太ㄍㄡˊ啦🤫🍜，#ramen #japanesefood #instafood #foodie #taipei #taiwan #yumyum
很多 0.28742399999999996
究竟 0.0
幸運 0.27024440000000005
濃厚 0.3
拉麵 0.15
沒 -0.1
客氣 0.0505

comment: 我就簡單Po兩張照片示意一下。，濕冷天這種東西根本欠吃。，，#也欠胖，#八幡屋深煎七味青花椒胡麻花生擔擔麵，#coldday ，#ramen ，#foodies ，#foodporn
簡單 0.026052
示意 0.1694408
東西 0.0
根本 0.02277495
ramen 0.15
sent_score: 0.36826775

comment: #勝王，叫了手揉麵 口感很彈，但彈到我覺得好像不是吃麵，下次去試試看細麵好了，雞白湯好喝 ，但他招牌是三骨一魚 雞牛豬
覺得 0.02
好像 0.44268620000000003
不是 -0.7609348000000001
下次 0.1
白湯 0.15
sent_score: -0.048248600000000086

comment: 是否中山北最強拉麵🃏
最強 0.0632301
拉麵 0.15
🃏 0.083
sent_score: 0.2962301

comment: 嚼嚼，晚間，剛睡醒，，超想睡覺 ..😑 #薩索清雞湯手揉麵，#八幡屋深煎七味擔擔麵，#4碗
😑 -0.311
sent_score: -0.311

comment: 勝王，干物醬油拉麵
拉麵 0.15
sent_score: 0.15

comment: 擔擔麵😋
😋 0.25
sent_score: 0.25

comment: 這個好喝到亂七八九飛起來的湯頭究竟怎麼回事？早知道的話吃清湯麵就更美滿了⋯⋯
究竟 0.0
知道 0.529169
美滿 0.6894288000000001
sent_score: 1.2185978

comment: 第一次吃到正版的松露です，，可是前面那組客人真的極白爛，搞的這碗麵口味都變差了。排隊的時候解壓縮，進店又一直自拍聊天，吃完了又不肯走。這間店對這些行為都有明文規定，為什麼可以這麼不要臉啊。
第一次 0.036000533
可是 -0.17361849999999998
變差 -0.47832275
排隊 -0.1
一直 0.2729052
不肯 -0.2054674
可以 0.1551797
不要臉 -0.12794356699999998
sent_score: -0.6212667839999999

comment: 只記得松露，#勝王 #勝王拉麵 #拉麵

果然 0.1125365
已經 0.029755599999999997
還好 0.2
排隊 -0.1
❀ 0.4
黃湯 0.0
很讚 0.3
▽ 0.4
sent_score: 1.4607333000000002

comment: 午餐簡單吃，夠蝦
簡單 0.026052
sent_score: 0.026052

comment: 📍 つけ麺 五ノ神製作所，🔹 白帶魚拉麵 $320﻿，﻿，美味：🌕🌕🌕🌕，服務：🌕🌕🌕🌕🌗，環境：🌕🌕🌕🌕🌗，UScore: 7.0 (還不錯，美中不足的是麵條)，———————————————————﻿，つけ麺 五ノ神製作所，#市政府 #松菸 旁來自東京新宿的拉麵店，聽說店面是酒吧改的，沒看招牌還會誤以為是餐酒館😅，店內有吧檯也有桌位，，主打濃厚蝦沾麵，每週末有限定口味，，這週是白帶魚拉麵😂，頭一次聽到拿白帶魚去熬湯的，好奢侈，，湯喝起來和一般煮干魚介湯不同，，因為使用新鮮魚去熬煮，不像煮干那種腥味較濃較重，味道大概在魚鮮跟魚腥的臨界點，搭配招牌大三角筍乾、紫洋蔥、水蓮、昆布絲、金桔，筍乾讓人驚艷的超好吃❤️ 看照片一直以為那坨是魚肉，吃起來鹹鹹的想說是無花果，跟店員確認才知道是昆布絲，，叉燒是幾乎全瘦的胛心肉？ 帶有咬勁，，黑胡椒醃製過帶出肉質的鮮甜，，肉片都炙燒，但油脂太少完全沒有感覺🤔，，麵條，我覺得軟硬度偏軟，個人比較愛偏硬的麵條，所以麵條部分我覺得是最可惜的地方，且麵條鹼味滿重的，這個就不得不稱讚勝王的麵了😍，，整體來說，五之神製作所，是現今難得唯二的豚骨蝦湯，且每週都有不同變化的限定口味，下次再來試試招牌沾麵❤️，﻿，小🐷識：﻿，魚介(ぎょかい)，魚貝類，泛指使用魚製湯﻿，———————————————————﻿，🌍 ﻿台北市信義區忠孝東路四段553巷6弄6號，⌚️ 11:30-15:00，17:00-21:00 (二公休)，﻿，延伸閱讀：﻿，札幌一幻拉麵﻿，﻿，#台北 #美食 #台北美食 #taipei #taiwan #food #restaurant #popyummy #popdaily #yummy #yummyfood #foodporn #foodie #ramen #japan #日本 #ぎょかい #ラーメン
📍 0.111
🔹 0.133
拉麵 0.15
美味 0.6096725000000001


sent_score: 5.592478117000002

comment: 進擊的澱粉之旅  適合小食量的人去 我回程又到附近外帶一碗炒刀削  我最近兩周的食慾是怎麼了
適合 0.775
sent_score: 0.775

comment: 覺得油麵真的讓人驚艷 ，辣味增跟柚子胡椒覺得都很搭，下次還想再來，（前一次吃的拉麵也是清爽的 不過還是比較推油麵)，，#sobashinn柑橘
覺得 0.02
驚艷 0.2573937
柚子 0.0
覺得 0.02
下次 0.1
拉麵 0.15
不過 0.0
還是 0.1
柑橘 0.0
sent_score: 0.6473937000000001

comment: 今週も一週間お疲れ様でした！，日本は祝日挟んだので少し楽だったかな❓，写真は先週のラーメン動画で特集した，柑橘ハマグリ鷄白湯ラーメンです🍜，食欲の秋、美味しいもの食べて午後も頑張りましょう❣️，.，大家都吃飽了嗎？，這個拉麵是上星期影片裡介紹的，柑橘蛤蜊雞白湯麵耶😊，秋天有很多好吃的食物！，所以吃好吃的東西、下午也要加油哦💪🏻，.，#拉麵#🍜#台灣#台北#午餐#秋天#sobashin#aburasobashin#柑橘系#ラーメン#ラーメン好きな人と繋がりたい #ラーメンインスタグラマー #foodstagram#ramen#taiwan #台湾好き#台湾美食#台湾旅行#台湾好きな人と繋がりたい
❓ 0.068
特集 0.0
柑橘 0.0
白湯 0.15
🍜 0.15
食欲 0.0
美味 0.6096725000000001
拉麵 0.15
介紹 0.05
柑橘 0.0
白湯 0.15
😊 0.644
很多 0.28742399999999996
好吃 0.37684395
好吃 0.37684395
東西 0.0
加油 0.012536499999999999
💪 0.555
拉麵 0.15
🍜 0.15
柑橘 0.0
ramen 0.15
美食 0.16
sent_score: 4.190320900000001

comment: #台北大安 #台北美食 #拉麵，-，soba shinn&柑橘，地址：台北市大安區仁愛路四段228-6號，營業時間：，11:45–13:30 / 17:30–20:30 (週二休)，電話: 02-2755 6705，整體90分，


comment: 一個意外然後撿到寶。，麵不錯叉燒飯稍稍普通。，，#麵屋壹の穴 #台北拉麵 #東區拉麵 #台北拉麵推薦 #雞白湯拉麵
意外 0.0
不錯 0.2
普通 -0.08
拉麵 0.15
拉麵 0.15
拉麵 0.15
推薦 0.019754
白湯 0.15
拉麵 0.15
sent_score: 0.8897540000000002

comment: 想說正式營運有中午場，結果平日來也排一堆人。
想 0.005
正式 0.475
營運 0.0
sent_score: 0.48

comment: 👇，🍜肉盛豚雞煮干沾麵（大） $300，評分：9⃣️3⃣️，整碗最有記憶點的是麵條，帶有淡淡的麥香味，但儘管我點大碗，份量仍比預期少。，沾醬的煮干味非常重，雞豚的味道相較之下沒那麼強烈，整個沾醬的鹹度很夠，配著店家特製的麵條吃起來很剛好。，肉盛會有雞叉燒和豬叉燒各三片，豬叉燒表現比較好，肥瘦均勻，沒有豬腥味，單吃或沾醬都適合。雞叉燒則強在口感非常軟嫩，味道較單薄，建議沾著醬吃。，糖心蛋的熟度剛剛好，滷的也有入味，增添口感上的稠度。，======================，🚶🏻再訪率：中上，價格跟大多數拉麵相差不遠，但麵的份量少了些，不過很好吃，也就想再找朋友來吃。店員服務都很親切，環境空間明亮，位子雖不到寬敞，但也不會壅塞感到不舒服。臉書上偶爾會推出不一樣的口味，看到有興趣的口味再來試試。，======================，麵屋壹之穴ichi-沾麵專門店，🧭 台北市大安區復興南路二段350號，📞 02-23776326（不接受預約），⏳ 1200-1400；1700-2000（日公休），======================，附菜單網誌版：http://jrrawlin.pixnet.net/blog/post/316365450，.，.，.，.，.，.，.，.，#大安區 #大安區美食 #118巷#118巷美食 #台大 #台大美食 #台大後門 #科技大樓 #科技大樓美食 #科技大樓站 #科技大樓站美食 #拉麵 #大安區拉麵 #沾麵 #大安區沾麵 #麵屋壹之穴 #麵屋壹之穴ichi #麵屋壹之穴沾麵專門店
👇 0.247
🍜 0.15
沾麵 0.15
最有 0.4004119
預期 0.1487412
非常 0.0788721
味道 0.

In [128]:
#====== get rank : hotpot ======
print('====== Hotpot rank ======')
store_rank = []
temp_rating = all_store_rating[:]
temp_rating.sort(reverse = True)
cnt = 1
for name,i in zip(store_names[1:],all_store_rating):
    rank = temp_rating.index(i)+1
    store_rank.append(rank)
    print('第'+str(cnt)+'名:',name,i,rank)
    cnt += 1
    

    
#====== MSE ======
def MSE(y, y_pred):
 
    mse = np.mean((y - y_pred)**2)
   
    return mse

print('\nstore_rank:',store_rank)
print('MSE:',MSE(np.array(range(0,10)),np.array(store_rank)))
print('r2_score:',r2_score(np.array(range(0,10)),np.array(store_rank)))
print('-------------------------------------')
print('Google_rank: [9,2,5,4,3,7,1,8,6,10]')
print('Google_MSE:',MSE(np.array(range(0,10)),np.array([9,2,5,4,3,7,1,8,6,10])))
print('Google_r2_score:',r2_score(np.array(range(0,10)),np.array([9,2,5,4,3,7,1,8,6,10])))

====== Hotpot rank ======
第1名: 辛殿麻辣鍋 0.7197687980639518 3
第2名: 海底撈火鍋 0.7112354059740985 4
第3名: 老四川巴蜀麻辣燙-南京店 0.6304563806839624 7
第4名: 這一鍋皇室秘藏鍋物 0.6733346068147888 5
第5名: 詹記麻辣火鍋 敦南店 0.5899759673788846 8
第6名: 新馬辣武昌店_新馬辣經典馬辣鍋 0.5138251770630505 9
第7名: 青花驕 麻辣鍋 0.8318692422505327 1
第8名: 馬辣頂級麻辣鴛鴦火鍋 漢口店 0.5115873664699272 10
第9名: 問鼎麻辣鍋忠孝店 0.6540240611774751 6
第10名: 滿堂紅bellavita店_松江店 0.7199912063061299 2

store_rank: [3, 4, 7, 5, 8, 9, 1, 10, 6, 2]
MSE: 16.6
r2_score: -1.0121212121212122
-------------------------------------
Google_rank: [9,2,5,4,3,7,1,8,6,10]
Google_MSE: 12.8
Google_r2_score: -0.5515151515151515


In [132]:
#====== get rank : ramen ======
print('====== Ramen ======')
store_rank = []
temp_rating = all_store_rating[:]
temp_rating.sort(reverse = True)
cnt = 1
for name,i in zip(store_names[1:],all_store_rating):
    rank = temp_rating.index(i)+1
    store_rank.append(rank)
    print('第'+str(cnt)+'名:',name,i,rank)
    cnt += 1
    

    
#====== MSE ======
def MSE(y, y_pred):
 
    mse = np.mean((y - y_pred)**2)
   
    return mse

print('\nstore_rank:',store_rank)
print('MSE:',MSE(np.array(range(1,10)),np.array(store_rank)))
print('r2_score:',r2_score(np.array(range(1,10)),np.array(store_rank)))
print('-------------------------------------')
print('Google_rank: [9,3,5,4,1,6,7,2,8]')
print('Google_MSE:',MSE(np.array(range(1,10)),np.array([9,3,5,4,1,6,7,2,8])))
print('Google_r2_score:',r2_score(np.array(range(1,10)),np.array([9,3,5,4,1,6,7,2,8])))

====== Ramen ======
第1名: 鬼金棒_中山北路 0.6962326955986844 7
第2名: 雞吉君拉麵 0.933045603531786 1
第3名: 山嵐拉麵_公館店_台灣總店 0.6722511674113475 8
第4名: 你回來啦 Okaeri 拉麵 0.8915706979777054 3
第5名: 麵屋壹慶 0.8193721374015748 4
第6名: 勝王 0.7906018630633683 6
第7名: 五之神製作所 0.8014299354043659 5
第8名: 柑橘 Shinn 0.8955572947335314 2
第9名: 麵屋壹の穴 ICHI_麵屋壹の穴ichi-沾麵專門店 0.6680849742424243 9

store_rank: [7, 1, 8, 3, 4, 6, 5, 2, 9]
MSE: 11.555555555555555
r2_score: -0.7333333333333334
-------------------------------------
Google_rank: [9,3,5,4,1,6,7,2,8]
Google_MSE: 13.555555555555555
Google_r2_score: -1.0333333333333332


In [115]:
# google ramen rating


print('Google Rating MSE:')
print('Google_MSE_ramen:',MSE(np.array(range(1,10)),np.array([9,3,5,4,1,6,7,2,8])))
print('Google_MSE_hotpot:',MSE(np.array(range(0,10)),np.array([9,2,5,4,3,7,1,8,6,10])))

Google Rating MSE:
Google_MSE_ramen: 13.555555555555555
Google_MSE_hotpot: 12.8


In [103]:

print('ramen:',mean_squared_error(np.array(range(1,10)),np.array([9,3,5,4,1,6,7,2,8])))
print('hotpot:',mean_squared_error(np.array(range(0,10)),np.array([9,2,5,4,3,7,1,8,6,10])))

ramen: 13.555555555555555
hotpot: 12.8


In [102]:

print('new rank ramen:',r2_score(np.array(range(1,10)),np.array([7, 1, 8, 3, 4, 6, 5, 2, 9])))
print('google ramen:',r2_score(np.array(range(1,10)),np.array([9,3,5,4,1,6,7,2,8])))

new rank ramen: -0.7333333333333334
google ramen: -1.0333333333333332


In [114]:
print('new rank ramen:',r2_score(np.array(range(1,10)),np.array([1, 4, 6, 3, 2, 8, 7, 5, 9])))

new rank ramen: 0.4


In [41]:
# store_names = ['辣麻味噌拉麵 鬼金棒','雞吉君拉麵','山嵐拉麵','你回來啦 Okaeri 拉麵']
# store_page_xpath = ['href="/explore/locations/541098880/"','href="/explore/tags/雞吉君拉麵/"','href="/explore/locations/610378299165578/"','href="/explore/locations/1009572868/okaeri/"']
# first_page_xpath = ['href="/p/B-Rf46-jXl_/"','href="/p/B-CEZjVl8GC/"','href="/p/B-Wy4mhnnvk/"', 'href="/p/B-KNu4oFwcv/"']
# get_page_count = [500,130,500,500]


# store_names = ['一蘭拉麵','麵屋壹の穴 ICHI','勝王']
# store_page_xpath = ['href="/explore/locations/255951018612300/"','href="/explore/locations/1719618628292660/ichi/"','href="/explore/locations/2062844753939787/"']
# first_page_xpath = ['href="/p/B-ZW6yUASZL/"','href="/p/B-ZXrQEnbZ-/"','href="/p/B-Zdt_rAfae/"']
# get_page_count = [1200,1000,1000]


# store_names = ['一蘭拉麵']
# store_page_xpath = ['href="/explore/locations/255951018612300/"']
# first_page_xpath = ['href="/p/B-ZW6yUASZL/"']
# get_page_count = [4000]


# store_names = ['柑橘 Shinn','五之神製作所 台灣','麵屋壹慶','勝王']
# store_page_xpath = ['href="/explore/locations/765135860311072/shinn/"','href="/explore/locations/1919809258303714/"','href="/explore/locations/308926743186267/"','href="/explore/locations/2062844753939787/"']
# first_page_xpath = ['href="/p/B-ZpZrvnZru/"','href="/p/B-Y78zMBrmj/"','href="/p/B-ZM0FnJglR/"','href="/p/B-Zdt_rAfae/"']
# get_page_count = [600,700,700,700]

# store_names = ['五之神製作所 台灣','麵屋壹慶','勝王']
# store_page_xpath = ['href="/explore/locations/1919809258303714/"','href="/explore/locations/308926743186267/"','href="/explore/locations/2062844753939787/"']
# first_page_xpath = ['href="/p/B-Y78zMBrmj/"','href="/p/B-ZM0FnJglR/"','href="/p/B-Zdt_rAfae/"']
# get_page_count = [700,700,700]

store_selenium_param = get_selenium_param('selenium_url')
store_names = []
store_page_xpath = []
first_page_xpath = []

for line in store_selenium_param:
    store_names.append(line[0])
    store_page_xpath.append(line[1])
    first_page_xpath.append(line[2])

    
print(first_page_xpath)
# store_names = [store_selenium_param[0][0]]
# store_page_xpath = [store_selenium_param[0][1]]
# first_page_xpath = [store_selenium_param[0][2]]
# get_page_count = [4000]
# print(first_page_xpath)

['href="/p/B9PFYDcFVFP/"', 'href="/p/B_Gyht1FSWV/"']


In [40]:
#===== scrape url from assigned date ======
startTime = time.time()

#====== use selenium to crawl store url ======
from selenium import webdriver

ig_home = "https://instagram.com"
driver = webdriver.Chrome(executable_path="chromedriver_linux64/chromedriver") # Use Chrome
driver.get(ig_home) # open ig home page
time.sleep(2)

# login
driver.find_element_by_xpath('//input[@name="username"]').send_keys('bestronger10142008')
driver.find_element_by_xpath('//input[@name="password"]').send_keys('igcoke1231')
time.sleep(2)
driver.find_element_by_xpath('//button[@type="submit"]').click()
time.sleep(5)

# iterate to get each store url
for store_index in range(len(store_names)):
    # input store name
    driver.find_element_by_xpath('//input[@type="text"]').send_keys(store_names[store_index]) 
    time.sleep(4)
    
    # driver.find_element_by_xpath('//a[@href="/explore/tags/雞吉君拉麵/"]').click() # click store page
    driver.find_element_by_xpath('//a[@' + store_page_xpath[store_index] + ']').click() # click store page
    time.sleep(5)
    
    # driver.find_element_by_xpath('//a[@href="/p/B-CEZjVl8GC/"]').click() # click first page link
    driver.find_element_by_xpath('//a[@' + first_page_xpath[store_index] + ']').click() # click first page link
    time.sleep(4)

    # get url of current page
    url_list, post_date, output_post_date = [], [], []
    post_date.append([driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text]) # get first page date

    # get 500 page url
    for i in range(5000):
        
        stopDate = post_date[-1][0]
        
        if stopDate[:stopDate.find('年')] != '2019':
#             print(stopDate)
            
            if check_exists_by_xpath('//a[@tabindex="0"][2]'):
                driver.find_element_by_xpath('//a[@tabindex="0"][2]').click() # click next page
                time.sleep(3)
            else:
                break
            time.sleep(1)
            post_date.append([driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text])
            continue
            
            
#         time.sleep(1)
        url_list.append([driver.current_url])
        time.sleep(1)
        temp_date = [driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text]
        post_date.append(temp_date)
        output_post_date.append(temp_date)
        
        if check_exists_by_xpath('//a[@tabindex="0"][2]'):
            driver.find_element_by_xpath('//a[@tabindex="0"][2]').click() # click next page
            time.sleep(4)
        else:
            break

        # stop date = 2019.8.1
        if stopDate[:stopDate.find('年')] == '2019' and stopDate[stopDate.find('年')+1:stopDate.find('月')] == '7':
            break
        
            
    ActionChains(driver).send_keys(Keys.ESCAPE).perform() # close floating page
    output_csv('dataset/ig_article/store_url/url_' + store_names[store_index], url_list[:-1], 'w+')
    output_csv('dataset/ig_article/store_url/date_' + store_names[store_index], output_post_date[:-1], 'w+')
    print(store_names[store_index]+'finished!')
    
driver.close()
stopTime = time.time()
print((stopTime-startTime)/60)

新馬辣武昌店finished!
新馬辣經典馬辣鍋finished!
青花驕麻辣鍋板橋縣民大道店finished!
馬辣頂級麻辣鴛鴦火鍋-中山店finished!
馬辣頂級麻辣鴛鴦火鍋 漢口店finished!
問鼎麻辣鍋忠孝店finished!
問鼎 麻辣鍋 養生鍋 粉絲俱樂部finished!
滿堂紅-松江店finished!
滿堂紅bellavita店finished!
153.04108297030132


In [43]:
#===== scrape url from first post ======
startTime = time.time()

#====== use selenium to crawl store url ======
from selenium import webdriver

ig_home = "https://instagram.com"
driver = webdriver.Chrome(executable_path="chromedriver_linux64/chromedriver") # Use Chrome
driver.get(ig_home) # open ig home page
time.sleep(2)

# login
driver.find_element_by_xpath('//input[@name="username"]').send_keys('bestronger10142008')
driver.find_element_by_xpath('//input[@name="password"]').send_keys('igcoke1231')
time.sleep(2)
driver.find_element_by_xpath('//button[@type="submit"]').click()
time.sleep(5)

# iterate to get each store url
for store_index in range(len(store_names)):
    # input store name
    driver.find_element_by_xpath('//input[@type="text"]').send_keys(store_names[store_index]) 
    time.sleep(5)
    
    # driver.find_element_by_xpath('//a[@href="/explore/tags/雞吉君拉麵/"]').click() # click store page
    driver.find_element_by_xpath('//a[@' + store_page_xpath[store_index] + ']').click() # click store page
    time.sleep(5)
    
    # driver.find_element_by_xpath('//a[@href="/p/B-CEZjVl8GC/"]').click() # click first page link
    driver.find_element_by_xpath('//a[@' + first_page_xpath[store_index] + ']').click() # click first page link
    time.sleep(3)

    # get url of current page
    url_list, post_date = [], []
    url_list.append([driver.current_url]) # get first page url
    time.sleep(1)
    post_date.append([driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text])
    time.sleep(3)

    # get 500 page url
#     flag = 0
#     for i in range(get_page_count[store_index]):
    for i in range(5000):
        if check_exists_by_xpath('//a[@tabindex="0"][2]'):
            driver.find_element_by_xpath('//a[@tabindex="0"][2]').click()
        else:
            break

        time.sleep(1)
        url_list.append([driver.current_url])
        time.sleep(5)
        post_date.append([driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text])
        stopDate = post_date[-1][0]
#         print(post_date[-1][0],type(post_date[-1][0]))
        
        # stop date = 2019.8.1
        if stopDate[:stopDate.find('年')] == '2019' and stopDate[stopDate.find('年')+1:stopDate.find('月')] == '7':
            break
            
    ActionChains(driver).send_keys(Keys.ESCAPE).perform() # close floating page
    output_csv('dataset/ig_article/store_url/url_' + store_names[store_index], url_list, 'w+')
    output_csv('dataset/ig_article/store_url/date_' + store_names[store_index], post_date, 'w+')
    print(store_names[store_index]+'finished!')
    
driver.close()
stopTime = time.time()
print(stopTime-startTime)

新馬辣武昌店finished!
青花驕 麻辣鍋finished!
7809.983314990997


In [259]:
# teststr = '2019年8月10日'
teststr = '3月23日'
if teststr[:teststr.find('年')] == '2019' and teststr[teststr.find('年')+1:teststr.find('月')] == '8':
    print('yes')

# extend dictionary

In [413]:
# extend dictionary
word_to_weight = {
    "土地公": 1,
    "土地婆": 1,
    "公有": 2,
    "緯來體育台": 1,
    "鬼金棒": 3,
    "海底撈": 5
}
dictionary = construct_dictionary(word_to_weight)
print(dictionary)

[(2, {'公有': 2.0}), (3, {'土地公': 1.0, '土地婆': 1.0, '鬼金棒': 3.0, '海底撈': 5.0}), (5, {'緯來體育台': 1.0})]
